In [1]:
import os    
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import re
PATH = "C:\Program Files (x86)\chromedriver.exe"

In [2]:
driver = webdriver.Chrome(PATH)
url='https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once'
driver.get(url)

<ipython-input-2-65110014fa9c>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [4]:
# getting the username and password
text_file = open("keys.txt", "r")
keys = text_file.read().splitlines()
text_file.close()

In [5]:
sign_in=driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/div/header/div[1]/div/ul/li[1]/a')
sign_in.click()                      
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='user[email]']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='user[password]']")))

username.clear()
username.send_keys(keys[0])
password.clear()
password.send_keys(keys[1])
time.sleep(3)
log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='submit']")))
log_in.click()

In [54]:
reviews = [] #good
review_rate = [] #good
review_date = [] 
rating = [] 
num_of_ratings = []
num_of_reviews = []
num_of_pages = []
published_year = []
published_month = []
author_followers = []
cover_type = []
genre = []

rev_links=[]

In [ ]:
page_url='https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page='
pages=list(range(1,101))
for page in pages:
    time.sleep(5)
    driver.get(page_url+str(page))
    print('num of page',page)
    try:
        table=driver.find_element(By.CSS_SELECTOR,'#all_votes > table > tbody ')
        rows=table.find_elements(By.TAG_NAME,'tr')
        print(len(rows))
    except:
        driver.refresh()
        time.sleep(5)
        table=driver.find_element(By.CSS_SELECTOR,'#all_votes > table > tbody ')
        rows=table.find_elements(By.TAG_NAME,'tr')
        print(len(rows))
        
    # getting all the book links from current page
    links=[]
    for row in rows:
        link=row.find_element(By.TAG_NAME,'a')
        links.append(link.get_attribute('href'))
    
    # running over all the book links
    for link in links:
        try:
            time.sleep(5)
            driver.get(link)
            time.sleep(5)   
            
            # book total rate
            try:
                rate=driver.find_element(By.XPATH,f"//span[@itemprop='ratingValue']").text
                print(rate)
            except:
                rate=np.nan

            # book pages
            try:
                pagesNumber =driver.find_element(By.XPATH,f"//span[@itemprop='numberOfPages']").text.split(" ")[0]
                print(pagesNumber)
            except:
                pagesNumber=np.nan

            # ratings number
            try:
                ratingsNumber =driver.find_element(By.XPATH,f"//meta[@itemprop='ratingCount']").get_attribute('content')
                print(ratingsNumber)
            except:
                ratingsNumber=np.nan
                
                
            # reviews number
            try:
                reviewsNumber =driver.find_element(By.XPATH,f"//meta[@itemprop='reviewCount']").get_attribute('content')
                print(reviewsNumber)
            except:
                reviewsNumber=np.nan

            # book genre
            try:
                genre_table=driver.find_element(By.CSS_SELECTOR,"body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.rightContainer > div:nth-child(6) > div > div.bigBoxBody > div ")
                genres=genre_table.find_elements(By.CLASS_NAME,"elementList")
                top_genre=genres[0].find_element(By.CLASS_NAME,'left').text.split(' ')[0]
                print(top_genre)
            except:
                top_genre= np.nan
    
            # book published date
            try:
                d= driver.find_element(By.ID,'details') 
                date_str=d.find_elements(By.CLASS_NAME,'row')[1].text.split(' ')
                string=""
                for w in date_str:
                    string+=w+' '
                year_pat='\d\d\d\d'
                year_pat2='\d\d\d'
                mon_pat="January|February|March|April|May|June|July|August|September|October|November|December"
                try:
                    month=re.findall(mon_pat,string)[0]
                    print(month)
                except:
                    month=np.nan
                try:
                    try:
                        year=re.findall(year_pat,string)[0]
                        print(year)
                    except:
                        year2=re.findall(year_pat2,string)[0]
                        print(year2)
                except:
                    year=np.nan

            except:
                year=np.nan
                month=np.nan

            # book cover
            try:
                cover =driver.find_element(By.XPATH,f"//span[@itemprop='bookFormat']").text
                print(cover)
            except:
                cover=np.nan
                
            # book author followers
            try:
                followers=driver.find_element(By.CLASS_NAME,"bookAuthorProfile__followerCount").text.split(' ')[0]
                print(followers)
            except:
                followers=np.nan

            # getting all the reviews
            try:
                r=driver.find_element(By.CLASS_NAME,'leftContainer')
                reviews_table=r.find_element(By.ID,'bookReviews')
                reviews = reviews_table.find_elements(By.CLASS_NAME,'friendReviews')
            except:
                continue
                
                
            # running over all reviews
            for review in reviews:
                
                # getting the full review link
                rev_link=review.find_element(By.TAG_NAME,'link').get_attribute('href')
                rev_links.append(rev_link)
                print('------------------------------------')
                print(rev_link)
                print('------------------------------------')

                # getting the rate of the review
                try:
                    stars = review.find_element(By.CLASS_NAME,'staticStars').get_attribute('title')
                    review_rate.append(stars)
                    print(stars)
                except:
                    review_rate.append(np.nan)
                    
                # getting the review date
                try:
                    r_date=review.find_element(By.CLASS_NAME,'reviewDate').text
                    review_date.append(r_date)
                    print(r_date)
                except:
                    review_date.append(np.nan)
                
                rating.append(rate)
                num_of_pages.append(pagesNumber)
                num_of_ratings.append(ratingsNumber)
                num_of_reviews.append(reviewsNumber)
                genre.append(top_genre)
                published_month.append(month)
                try:
                    published_year.append(year)
                except:
                    published_year.append(year2)
                cover_type.append(cover)
                author_followers.append(followers)

        except:
            driver.refresh()


In [ ]:
df= pd.DataFrame({'Reviews_link':rev_links,'Review_date':review_date, 'Review_rate':review_rate,'Ratings_number':num_of_ratings,'Reviews_number':num_of_reviews,'Total_rate':rating, 'Pages':num_of_pages,'Year_of_book':published_year, 'Month_of_book':published_month, 'Author_followers':author_followers, 'Cover':cover_type ,'Genre':genre})
df.to_csv('Reviews_updated.csv',index=False)
df

In [106]:
# I got help with the reviews links crawling and because of lack of time I minimized my dataframe to 150000 rows
df.dropna(inplace=True)
new_df=df[:150000]

All_links=new_df['Reviews_link'].tolist()
links1=All_links[:25000] #success
links2=All_links[25000:50000] #success
links3=All_links[50000:75000] #success
links4=All_links[75000:100000] #24,846 success
links5=All_links[100000:125000] #fail
links6=All_links[125000:] #success

In [ ]:
reviews = []
i=1
for l in links1:
    try:
        driver.get(l)
        time.sleep(3)
        
        try:
            text=driver.find_element(By.CLASS_NAME,'reviewText').text
            reviews.append(text)
            print('----',i,' / ',length,'------')
            print(text)
            print('----------------------')
        except:
            reviews.append(np.nan)
        i+=1
    except:
        driver.refresh()

to_save=pd.DataFrame({'reviews':reviews})  

to_save.to_csv('Try1.csv')

In [ ]:
reviews = []
i=1
for l in links2:
    try:
        driver.get(l)
        time.sleep(3)
        
        try:
            text=driver.find_element(By.CLASS_NAME,'reviewText').text
            reviews.append(text)
            print('----',i,' / ',length,'------')
            print(text)
            print('----------------------')
        except:
            reviews.append(np.nan)
        i+=1
    except:
        driver.refresh()

to_save=pd.DataFrame({'reviews':reviews})  

to_save.to_csv('Try2.csv')

In [ ]:
reviews = []
i=1
for l in links3:
    try:
        driver.get(l)
        time.sleep(3)
        
        try:
            text=driver.find_element(By.CLASS_NAME,'reviewText').text
            reviews.append(text)
            print('----',i,' / ',length,'------')
            print(text)
            print('----------------------')
        except:
            reviews.append(np.nan)
        i+=1
    except:
        driver.refresh()

to_save=pd.DataFrame({'reviews':reviews})  

to_save.to_csv('Try3.csv')

In [ ]:
reviews = []
i=1
for l in links4:
    try:
        driver.get(l)
        time.sleep(3)
        
        try:
            text=driver.find_element(By.CLASS_NAME,'reviewText').text
            reviews.append(text)
            print('----',i,' / ',length,'------')
            print(text)
            print('----------------------')
        except:
            reviews.append(np.nan)
        i+=1
    except:
        driver.refresh()

to_save=pd.DataFrame({'reviews':reviews})  

to_save.to_csv('Try4.csv')

In [ ]:
reviews = []
i=1
for l in links5:
    try:
        driver.get(l)
        time.sleep(3)
        
        try:
            text=driver.find_element(By.CLASS_NAME,'reviewText').text
            reviews.append(text)
            print('----',i,' / ',length,'------')
            print(text)
            print('----------------------')
        except:
            reviews.append(np.nan)
        i+=1
    except:
        driver.refresh()

to_save=pd.DataFrame({'reviews':reviews})  

to_save.to_csv('Try5.csv')

In [ ]:
reviews = []
i=1
for l in links6:
    try:
        driver.get(l)
        time.sleep(3)
        
        try:
            text=driver.find_element(By.CLASS_NAME,'reviewText').text
            reviews.append(text)
            print('----',i,' / ',length,'------')
            print(text)
            print('----------------------')
        except:
            reviews.append(np.nan)
        i+=1
    except:
        driver.refresh()

to_save=pd.DataFrame({'reviews':reviews})  

to_save.to_csv('Try6.csv')

In [ ]:
new_df.drop(new_df[100000:125000].index,inplace=True)
new_df.drop(new_df[99846:100000].index,inplace=True)
new_df

In [108]:
s1=pd.read_csv('Try1.csv')
s2=pd.read_csv('Try2.csv')
s3=pd.read_csv('Try3.csv')
s4=pd.read_csv('Try4.csv')
s6=pd.read_csv('Try6.csv')

In [109]:
revs=[]

r1=s1['reviews'].tolist()
r2=s2['reviews'].tolist()
r3=s3['reviews'].tolist()
r4=s4['reviews'].tolist()
r6=s6['reviews'].tolist()

for item in r1:
    revs.append(item)
for item in r2:
    revs.append(item)
for item in r3:
    revs.append(item)
for item in r4:
    revs.append(item)
for item in r6:
    revs.append(item)
len(revs)

124846

In [ ]:
new_df.insert(0,'Reviews',revs)
new_df

In [99]:
new_df.to_csv('Complete_df.csv',index=False)